<a href="https://colab.research.google.com/github/sdariza/tesis-cnn/blob/master/Plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna
!pip install netCDF4
!pip install basemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 860.6/860.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 32.0 MB/s eta 0:00:00
  Attempting uninstall: pyproj
    Found existing installation: pyproj 3.6.0
    Uninstalling pyproj-3.6.0:
      Successfully uninstalled pyproj-3.6.0


In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import optuna
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

BATCHSIZE = 12
EPOCHS=100

In [3]:
def create_model(trial):
  kernel_sizes = [(2,2),(2,3),(3,2),(3,3),(2,4),(4,2),(3,4),(4,3),(4,4)]
  kz_selected = trial.suggest_categorical("kernel_size", kernel_sizes)
  alphas = [.5, .7, .9]
  alpha_selected = trial.suggest_categorical("alpha", alphas)
  model = tf.keras.models.Sequential(name=f'CNN-Weather-Forecasting-Optimizing-Parameters')
  model.add(tf.keras.layers.Input(shape=(181,360,1), name='input_layer'))
  model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=kz_selected, padding='same', name='conv2D_1'))
  model.add(tf.keras.layers.LeakyReLU(alpha=alpha_selected, name='act_1'))
  model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=kz_selected, padding='same', name='conv2D_2'))
  model.add(tf.keras.layers.LeakyReLU(alpha=alpha_selected, name='act_2'))
  model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2),padding='same', name='AvP_1'))
  model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=kz_selected, padding='same', name='conv2D_3'))
  model.add(tf.keras.layers.LeakyReLU(alpha=alpha_selected, name='act_3'))
  model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=kz_selected, padding='same', name='conv2D_4'))
  model.add(tf.keras.layers.LeakyReLU(alpha=alpha_selected, name='act_4'))
  model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2),padding='same', name='AvP_2'))

  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=kz_selected, padding='same', name='conv2D_5'))
  model.add(tf.keras.layers.LeakyReLU(alpha=alpha_selected, name='act_5'))

  model.add(tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=kz_selected, padding='same', name='conv2DT_1'))
  model.add(tf.keras.layers.LeakyReLU(alpha=alpha_selected, name='act_6'))
  model.add(tf.keras.layers.UpSampling2D(size=(2,2), name='UpS2D_1'))
  model.add(tf.keras.layers.Cropping2D(cropping=((1,0),(0,0)), name='Cropping_1'))
  model.add(tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=kz_selected, padding='same', name='conv2DT_2'))
  model.add(tf.keras.layers.LeakyReLU(alpha=alpha_selected, name='act_7'))
  model.add(tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=kz_selected, padding='same', name='conv2DT_3'))
  model.add(tf.keras.layers.LeakyReLU(alpha=alpha_selected, name='act_8'))
  model.add(tf.keras.layers.UpSampling2D(size=(2,2), name='UpS2D_2'))
  model.add(tf.keras.layers.Cropping2D(cropping=((1,0),(0,0)), name='Cropping_2'))
  model.add(tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=kz_selected, padding='same', name='conv2DT_4'))
  model.add(tf.keras.layers.LeakyReLU(alpha=alpha_selected, name='act_9'))
  model.add(tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=kz_selected, padding='same', name='output'))
  model.add(tf.keras.layers.LeakyReLU(alpha=alpha_selected, name='act_10'))
  model.summary()
  return model

In [4]:
def create_optimizer(trial):
  kwargs = {}
  kwargs["learning_rate"] = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
  optimizer = getattr(tf.optimizers, 'Adam')(**kwargs)
  return optimizer

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
X_temp = nc.Dataset('/content/drive/MyDrive/research/nc-files/t0.nc')
X = X_temp.variables['t'][:].data - 273.15
Y_temp = nc.Dataset('/content/drive/MyDrive/research/nc-files/t6.nc')
Y = Y_temp.variables['t'][:].data - 273.15

In [7]:
X_temp.close()
Y_temp.close()

In [8]:
def get_data():
    # Split the data into training, validation, and test sets
    split_ratio_train = 0.8
    split_ratio_val = 0.2
    num_samples = len(X)
    num_train = int(split_ratio_train * num_samples)
    num_val = int(split_ratio_val * num_samples)
    train_data = X[:num_train]
    val_data = X[num_train:num_train+num_val]
    train_labels = Y[:num_train]
    val_labels = Y[num_train:num_train+num_val]
    train_ds = tf.data.Dataset.from_tensor_slices(tensors=(train_data, train_labels))
    train_ds = train_ds.shuffle(60000).batch(BATCHSIZE)

    valid_ds = tf.data.Dataset.from_tensor_slices(tensors=(val_data, val_labels))
    valid_ds = valid_ds.shuffle(10000).batch(BATCHSIZE)

    return train_ds, valid_ds

In [9]:
def plot_loss_metric(history, params, _trial_id):
    # Obtener las métricas del entrenamiento y la validación
    train_loss = history.history['loss']
    train_mae = history.history['mean_absolute_error']
    val_loss = history.history['val_loss']
    val_mae = history.history['val_mean_absolute_error']

    # Graficar las métricas
    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(12, 4))
    kz, op, lr = params['kernel_size'], params['optimizer'], params['learning_rate']
    selected_arch = params['arch']
    plt.suptitle(f'trialId:{_trial_id} arch:{selected_arch} kz:{kz} op:{op}, lr:{lr}')

    plt.subplot(1, 2, 1)
    plt.plot(epochs, np.log(train_loss), label='Training Loss')
    plt.plot(epochs, np.log(val_loss), label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('$log(loss)$')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, np.log(train_mae), label='Training MAE')
    plt.plot(epochs, np.log(val_mae), label='Validation MAE')
    plt.xlabel('Epoch')
    plt.ylabel('$log(MAE)$')
    plt.title(f'Training and Validation MAE')
    plt.legend()

    plt.tight_layout()
    plt.savefig(f'optimizePlots/trial{_trial_id}arch{selected_arch}.png')

In [10]:
def objective(trial):
    train_ds, valid_ds = get_data()

    model = create_model(trial)
    optimizer = create_optimizer(trial)
    early_stop = EarlyStopping(monitor='val_loss', patience=10, mode='auto', verbose=1)
    model.compile(optimizer=optimizer,loss=tf.keras.losses.MeanSquaredError(), metrics=tf.keras.losses.MeanAbsoluteError())
    history = model.fit(train_ds, validation_data=valid_ds, epochs=EPOCHS, callbacks=[early_stop], verbose=1)

    plot_loss_metric(history,trial.__dict__['_cached_frozen_trial'].params,trial.__dict__['_trial_id'])

    mae = model.evaluate(valid_ds)[1]

    return mae

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-07-03 03:36:13,006] A new study created in memory with name: no-name-800c5b3f-90e7-4d8e-a755-65df171b6901
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 2) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 3) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (3, 2) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices f

Model: "CNN-Weather-Forecasting-Optimizing-Parameters"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2D_1 (Conv2D)           (None, 181, 360, 32)      288       
                                                                 
 act_1 (LeakyReLU)           (None, 181, 360, 32)      0         
                                                                 
 conv2D_2 (Conv2D)           (None, 181, 360, 32)      8224      
                                                                 
 act_2 (LeakyReLU)           (None, 181, 360, 32)      0         
                                                                 
 AvP_1 (AveragePooling2D)    (None, 91, 180, 32)       0         
                                                                 
 conv2D_3 (Conv2D)           (None, 91, 180, 64)       16448     
                                                                 
 act_3 (LeakyReLU)   